In [1]:
!kaggle datasets download -d  harishvutukuri/dogs-vs-wolves -p ../../data

dogs-vs-wolves.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
!unzip -q -u ../../data/dogs-vs-wolves.zip -d ../../data/dogs-vs-wolves

In [3]:
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile # TF temporal bugfix

In [4]:
%load_ext tensorboard

## Imports

In [5]:
import os

# Torch #
import torch
from torch import nn, optim
from torch.utils.data import DataLoader 

# Torchvision #
from torchvision import datasets, transforms, models

# Torchero #
import torchero
from torchero import SupervisedTrainer # Training the model
from torchero.callbacks.tensorboard import TensorBoardLogger
from torchero.callbacks import ProgbarLogger # Model supervising
from torchero.utils.data import train_test_split
from torchero.utils import show_imagegrid_dataset
from torchero.utils.tensorboard import SummaryWriter, write_imagegrid_dataset, write_model
from matplotlib import pyplot as plt

In [6]:
train_writer = SummaryWriter('runs/Dogs vs Wolves/Train')
val_writer = SummaryWriter('runs/Dogs vs Wolves/Test')

## Training dataset

In [7]:
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor()])

In [8]:
dataset = datasets.ImageFolder(root='../../data/dogs-vs-wolves/data', transform=transform)

In [9]:
train_ds, val_ds = train_test_split(dataset)

In [10]:
write_imagegrid_dataset(train_writer, 'Dataset', train_ds)

In [33]:
%tensorboard --logdir runs/

### Test dataset

In [12]:
write_imagegrid_dataset(val_writer, 'Dataset', val_ds)

### DataLoader

In [13]:
train_dl = DataLoader(train_ds, batch_size=10, num_workers=6, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=10, num_workers=6)

## Define model

In [14]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.weight.shape[-1], 1)
    
    def forward(self, x):
        x = self.model(x)
        x = x.squeeze()
        return x

In [15]:
model = Network()

In [16]:
write_model(train_writer, model, next(iter(train_dl))[0])

for param_group in trainer.optimizer.param_groups:
    print(param_group["lr"])

## Train model

In [17]:
trainer = SupervisedTrainer(model=model,
                            logging_frecuency=10,
                            optimizer='adam',
                            criterion='binary_cross_entropy_wl',
                            acc_meters=['recall_wl', 'precision_wl', 'binary_with_logits_accuracy', 'f1_wl'],
                            callbacks=[ProgbarLogger(notebook=True), TensorBoardLogger(train_writer, test_writer=val_writer)])

In [18]:
trainer.cuda()

In [19]:
trainer.train(train_dl, val_dl, epochs=10)

### Writting Embeddings to Projector

In [20]:
summary_writer = SummaryWriter('runs/Dogs vs Wolves')

In [21]:
class Identity(nn.Module):
    def forward(self, x):
        return x

In [22]:
fc = model.model.fc.cpu()

In [23]:
model.model.fc = Identity()

In [24]:
from itertools import islice
import numpy as np

In [25]:
trainer.cpu()

In [26]:
total_classes = []
embeddings = []
total_images = []
total_probas = []
transform = transforms.Compose([transforms.ToPILImage(), transforms.Resize((64, 64)), transforms.ToTensor()])
modes = []

dl = DataLoader(train_ds, batch_size=10, shuffle=True)

for ims, classes in islice(dl, 10):
    ims, classes = next(iter(dl))
    embs = model(ims)
    probas = torch.nn.functional.sigmoid(fc(embs)).squeeze(-1)
    total_probas.extend(probas)
    embeddings.append(embs)
    total_images.extend(transform(im) for im in ims)
    total_classes.extend(classes)
    modes.extend(['train']*len(classes))
    
dl = DataLoader(val_ds, batch_size=10, shuffle=True)

for ims, classes in islice(dl, 10):
    ims, classes = next(iter(dl))
    embs = model(ims)
    probas = torch.nn.functional.sigmoid(fc(embs)).squeeze(-1)
    total_probas.extend(probas)
    embeddings.append(embs)
    total_images.extend(transform(im) for im in ims)
    total_classes.extend(classes)
    modes.extend(['test']*len(classes))

/usr/lib/python3.8/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [28]:
classes = np.array(dataset.classes)
embeddings = torch.stack(embeddings, 0)
embeddings = embeddings.reshape(-1, 512)

total_images = torch.stack(total_images, 0)
total_classes = classes[torch.stack(total_classes)]
total_probas = torch.stack(total_probas)
predicted_classes = classes[(total_probas > 0.5).int()]

failed = np.where(total_classes == predicted_classes, 'ok', 'fail')

In [29]:
summary_writer.add_embedding(embeddings.cpu(),
                             metadata=list(zip(total_classes, modes, predicted_classes, total_probas, failed)),
                             label_img=total_images,
                             global_step=0,
                             tag='embeddings',
                             metadata_header=['label', 'mode', 'predicted_class', 'proba', 'correct'])

In [30]:
summary_writer.flush()

In [31]:
summary_writer.close()